In [24]:
from datetime import datetime
import pandas as pd
import requests
import json
import boto3
import sqlalchemy

In [25]:
! pip3 install kaggle

Defaulting to user installation because normal site-packages is not writeable


In [32]:
!kaggle datasets list -s 'Police Deaths America'

ref                                                 title                             size  lastUpdated          downloadCount  voteCount  usabilityRating  
--------------------------------------------------  -------------------------------  -----  -------------------  -------------  ---------  ---------------  
datasets/hakabuk/gun-deaths-in-the-us               Gun Deaths in the US: 2012-2014  715KB  2017-01-25 10:32:17           4902         64  0.7647059        
datasets/satoshidatamoto/police-deaths-in-americae  Police Deaths in America           1MB  2022-05-15 20:35:51             48          2  1.0              


In [11]:

    url = "https://apisidra.ibge.gov.br/values/t/6578/n1/all/v/10163/p/all/d/v10163%201?formato=json"
    data = requests.get(url)
    data = data.json() 

    #path = '/usr/local/airflow/'
    #filename = 'distritos.csv'
    
    df = pd.DataFrame(data)
    #df = json_normalize(df['municipio'])
    #df.to_csv(path+filename)

In [12]:
df.head()

,NC,NN,MC,MN,V,D1C,D1N,D2C,D2N,D3C,D3N
0,Nível Territorial (Código),Nível Territorial,Unidade de Medida (Código),Unidade de Medida,Valor,Brasil (Código),Brasil,Variável (Código),Variável,Ano (Código),Ano
1,1,Brasil,45,Pessoas,3.0,1,Brasil,10163,Número médio de moradores,2016,2016
2,1,Brasil,45,Pessoas,3.0,1,Brasil,10163,Número médio de moradores,2017,2017
3,1,Brasil,45,Pessoas,2.9,1,Brasil,10163,Número médio de moradores,2018,2018
4,1,Brasil,45,Pessoas,2.9,1,Brasil,10163,Número médio de moradores,2019,2019
